In [8]:
# Import modules, load environment, and set the model, max tokens, and create the client
from datetime import datetime, timedelta
from anthropic import Anthropic
from anthropic.types import Message
from IPython.display import Markdown
from dotenv import load_dotenv
import json

load_dotenv()

model = "claude-3-7-sonnet-latest"
num_tokens = 1000
client = Anthropic()

In [9]:
# Claude message helper functions
def add_user_message(messages, message):
   user_message = {
      "role": "user",
      "content": message.content if isinstance(message, Message) else message,
   }
   messages.append(user_message)

def add_assistant_message(messages, message):
   assistant_message = {
      "role": "assistant",
      "content": message.content if isinstance(message, Message) else message,
   }
   messages.append(assistant_message)

def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
   params = {
      "model": model,
      "max_tokens": num_tokens,
      "messages": messages,
      "temperature": temperature,
      "stop_sequences": stop_sequences,
   }

   if tools:
      params["tools"] = tools
   
   if system:
      params["system"] = system
   
   message = client.messages.create(**params)
   return message

def text_from_message(message):
   return "\n".join(
      [block.text for block in message.content if block.type == "text"]
   )

In [10]:
# Tools and their schemae

def get_current_datetime(date_format="%Y-%m-%d %H:%M:%S"):
   if not date_format:
      raise ValueError(f"{date_format} cannot be empty.")
   
   return datetime.now().strftime(date_format)

get_current_datetime_schema = {
   "name": "get_current_datetime",
   "description": "Get the current date and time formatted according to a specified pattern. Returns the current datetime as a formatted string.",
   "input_schema": {
     "type": "object",
     "properties": {
       "date_format": {
         "type": "string",
         "description": "Python strftime format string for datetime formatting. Common patterns: '%Y-%m-%d' for date only (e.g., 2025-10-13), '%H:%M:%S' for time only (e.g., 14:30:45), '%Y-%m-%d %H:%M:%S' for full datetime (e.g., 2025-10-13 14:30:45), '%B %d, %Y' for readable date (e.g., October 13, 2025). Cannot be empty. Defaults to '%Y-%m-%d %H:%M:%S' if not provided.",
         "default": "%Y-%m-%d %H:%M:%S"
       },
     },
     "required": []
   },
}

def add_duration_to_datetime(
      datetime_str, duration=0, unit="days", input_format="%Y-%m-%d"
):
   date = datetime.strptime(datetime_str, input_format)

   if unit == "seconds":
      new_date = date + timedelta(seconds=duration)
   elif unit == "minutes":
      new_date = date + timedelta(minutes=duration)
   elif unit == "hours":
      new_date = date + timedelta(hours=duration)
   elif unit == "days":
      new_date = date + timedelta(days=duration)
   elif unit == "weeks":
      new_date = date + timedelta(weeks=duration)
   elif unit == "months":
      month = date.month + duration
      year = date.year + month // 12
      month = month % 12
      if month == 0:
         month = 12
         year -= 1
      day = min(
         date.day,
         [
            31,
            29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28,
            31,
            30,
            31,
            30,
            31,
            31,
            30,
            31,
            30,
            31,
         ][month - 1],
      )
      new_date = date.replace(year=year, month=month, day=day)
   elif unit == "years":
      new_date = date.replace(year=date.year + duration)
   else:
      raise ValueError(f"Given unsupported/unrecognized time unit: {unit}")
   
   return new_date.strftime("%A, %B %d, %Y %I:%M:%S %p")

def set_reminder(content, timestamp):
   return print(f"----\nSetting the following reminder for {timestamp}:\n{content}\n----")

def batch_tool(invocations=[]):
   batch_output = []

   for invocation in invocations:
      name = invocation["name"]
      args = json.loads(invocation["arguments"])

      tool_output = run_tool(name, args)

      batch_output.append({
         "tool_name": name,
         "output": tool_output
      })
   
   return batch_output

add_duration_to_datetime_schema = {
  "name": "add_duration_to_datetime",
  "description": "Adds a specified duration to a datetime string and returns the resulting datetime in a detailed format. This tool converts an input datetime string to a Python datetime object, adds the specified duration in the requested unit, and returns a formatted string of the resulting datetime. It handles various time units including seconds, minutes, hours, days, weeks, months, and years, with special handling for month and year calculations to account for varying month lengths and leap years. The output is always returned in a detailed format that includes the day of the week, month name, day, year, and time with AM/PM indicator (e.g., 'Thursday, April 03, 2025 10:30:00 AM').",
  "input_schema": {
    "type": "object",
    "properties": {
      "datetime_str": {
        "type": "string",
        "description": "The starting datetime as a string. Must match the format specified in input_format parameter (default: YYYY-MM-DD, e.g., '2024-03-15')"
      },
      "duration": {
        "type": "integer",
        "description": "The amount of time to add. Can be positive (for future dates) or negative (for past dates). Defaults to 0.",
        "default": 0
      },
      "unit": {
        "type": "string",
        "enum": ["seconds", "minutes", "hours", "days", "weeks", "months", "years"],
        "description": "The unit of time for the duration. Options: 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', or 'years'. Default is 'days'.",
        "default": "days"
      },
      "input_format": {
        "type": "string",
        "description": "The format of the input datetime string using Python strftime codes (e.g., '%Y-%m-%d' for YYYY-MM-DD, '%Y-%m-%d %H:%M:%S' for datetime with time). Default is '%Y-%m-%d'.",
        "default": "%Y-%m-%d"
      }
    },
    "required": ["datetime_str"]
  }
}

set_reminder_schema = {
   "name": "set_reminder",
   "description": "Creates a timed reminder that will notify the user at the specified time with the provided content.  This tools schedules a notification to be delivered to the user at the exact timestamp provided.  It should be used when a user wants to be reminded about something specific at a future point in time.  The reminder system will store the content and timestamp, then trigger a notification through the user's preferred notification channels (mobile phone alerts, SMS, email, etc.) when the specified time arrives.  Reminders are permitted even if the application is closed or the device is restarted. Users can rely on this function for important time-sensitive notifications such as meetings, tasks, medication schedules, or any other time-bound activities.",
   "input_schema": {
      "type": "object",
      "properties": {
         "content": {
            "type": "string",
            "description": "The message text that will be displayed in the reminder notification. This should contain the specific information the user wants to be reminded about, such as 'Take medication', 'Join video call with team', or 'Pay utility bills'.",
         },
         "timestamp": {
            "type": "string",
            "description": "The exact date and time when the reminder should be triggered, formatted as an ISO 8601 timestamp (YYYY-MM-DDTHH:MM:SS) or a Unix timestamp. The system handles all timezone processing internally, ensuring reminders are triggered at the correct time regardless of where the user is located. Users can simply specify the desired time without worrying about timezone configurations.",
         },
      },
      "required": ["content", "timestamp"],
   },
}

batch_tool_schema = {
   "name": "batch_tool",
   "description": "Invoke multiple other tool calls simultaneously.",
   "input_schema": {
      "type": "object",
      "properties": {
         "invocations": {
            "type": "array",
            "description": "The tool calls to invoke",
            "items": {
               "type": "object",
               "properties": {
                  "name": {
                     "type": "string",
                     "description": "The name of the tool to invoke"
                  },
                  "arguments": {
                     "type": "object",
                     "description": "The arguments to pass to the tool"
                  }
               }
            }
         }
      }
   }
}

In [11]:
def run_tool(tool_name, tool_input):
   if tool_name == "get_current_datetime":
      return get_current_datetime(**tool_input)
   elif tool_name == "add_duration_to_datetime":
      return add_duration_to_datetime(**tool_input)
   elif tool_name == "set_reminder":
      return set_reminder(**tool_input)
   elif tool_name == "batch_tool":
      return batch_tool(**tool_input)

In [15]:
def run_tools(message):
   tool_requests = [
      block for block in message.content if block.type == "tool_use"
   ]
   tool_result_blocks = []

   for tool_request in tool_requests:
      try:
         tool_output = run_tool(tool_request.name, tool_request.input)
         tool_result_block = {
            "type": "tool_result",
            "tool_use_id": tool_request.id,
            "content": json.dumps(tool_output),
            "is_error": False
         }
      except Exception as e:
         tool_result_block = {
            "type": "tool_result",
            "tool_use_id": tool_request.id,
            "content": f"Error: {e}",
            "is_error": True
         }
      
      tool_result_blocks.append(tool_result_block)
   
   return tool_result_blocks

In [16]:
def run_conversation(messages):
   while True:
      response = chat(messages, tools=[
         get_current_datetime_schema,
         add_duration_to_datetime_schema,
         set_reminder_schema,
         batch_tool_schema
      ])
      add_assistant_message(messages, response)
      print(text_from_message(response))

      if response.stop_reason != "tool_use":
         break

      tool_results = run_tools(response)
      add_user_message(messages, tool_results)

   return messages

In [17]:
messages = []
add_user_message(
   messages,
   "Set a reminder for my Dentist's appointment 15 days from Nov 2nd, 2025.",
)
run_conversation(messages)

I'll help you set a reminder for your dentist appointment. Let me calculate the date that is 15 days from November 2nd, 2025, and then set the reminder for you.
Now I'll set the reminder for your dentist appointment on November 17, 2025:
----
Setting the following reminder for 2025-11-17T00:00:00:
Dentist's appointment
----
I've set a reminder for your dentist's appointment on Monday, November 17, 2025. You'll receive a notification about your appointment at midnight on that day. Is there any specific time for your appointment that you'd like me to update in the reminder?


[{'role': 'user',
  'content': "Set a reminder for my Dentist's appointment 15 days from Nov 2nd, 2025."},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll help you set a reminder for your dentist appointment. Let me calculate the date that is 15 days from November 2nd, 2025, and then set the reminder for you.", type='text'),
   ToolUseBlock(id='toolu_01VRctvTAyu3awHxTWi9HUQq', input={'datetime_str': '2025-11-02', 'duration': 15, 'unit': 'days'}, name='add_duration_to_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01VRctvTAyu3awHxTWi9HUQq',
    'content': '"Monday, November 17, 2025 12:00:00 AM"',
    'is_error': False}]},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="Now I'll set the reminder for your dentist appointment on November 17, 2025:", type='text'),
   ToolUseBlock(id='toolu_01MYdP4kF81g9g8E8mE5QB79', input={'content': "Dentist's appointment", 'timestamp': '2025-1